In [30]:
import os, re, datetime
import pandas as pd
from collections import OrderedDict 
from docxtpl import DocxTemplate

In [31]:
df = pd.read_excel('胃癌化疗用药数据库20190830.xlsx')
df = df.dropna(axis = 0, subset = ['类别'] )
#填充nan值的函数
def make_column(string):
    a = ''
    list1 = []
    for i in df.index:
        if type(df[string][i]) != type(df[string][2]):
            a = df[string][i]
            list1.append(a)
        else:
            list1.append(a)
    return list1

for i in ['化疗药物', '基因', '位点', '证据等级']:
    df[i] = (make_column(i))

In [32]:
dic = {}
dic1 = {}
for i in df.index:
    dic.setdefault(df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i], {})
    dic1.setdefault(df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i], {})

    dic[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]][df['基因型'][i]] = df['解释'][i]
    dic1[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]][df['基因型'][i]] = df['类别'][i]
    dic1[df['化疗药物'][i] + '\t' + df['基因'][i] + '\t' + df['位点'][i]]['证据等级'] = df['证据等级'][i]

In [33]:
list1 = ['rs4148323', 'rs67376798', 'rs1800566']
list2 = ['rs25487', 'rs11615', 'rs662']

list3 = []

In [34]:
def modify_result(df_name, list1, list2, list3):
    #list1:基因型与数据库相反的位点列表，list2:基因型与数据库反向互补的位点列表 
    #list3:基因型与数据库互补的位点列表
    reverse_complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()[::-1]
    complement = lambda gt: gt.replace('A','t').replace('T','a').replace('G','c').replace('C','g').upper()
    for i in range(df_name.shape[0]):
        for line1 in list1:
            df_name.at[i, line1] = df_client.at[i, line1][::-1]#将检测结果中与数据库相反的基因型校正
        for line2 in list2:
            df_name.at[i, line2] = reverse_complement(df_name.at[i, line2])
        for line3 in list3:
            df_name.at[i, line3] = complement(df_name.at[i, line3])

In [35]:
df_client = pd.read_excel('胃癌检测结果.xlsx')
modify_result(df_client, list1, list2, list3)
list_rs = re.findall(r'rs\d+', str(df_client.columns))
dic_client = {}
for i in range(df_client.shape[0]):
    for j in list_rs:
        dic_client.setdefault (df_client['姓名'][i], {})
        dic_client[df_client['姓名'][i]][j] = df_client[j][i]

In [36]:
level_dic = {'1A':6 , '1B':5, '2A':4, '2B':3, '3':2, '4':1 }
#输入客户名字，产生临床提示结果
def clinical_prompt(client_name ):
    d_result = {}#临床提示映射
    list_pinggu = []
    d_test = {}
    dic_aa = {}
    for i in dic1.keys():
        list_pinggu.append(dic1[i][dic_client[client_name][i.split('\t')[2]]] + '--' + str(dic1[i]['证据等级']) )
        dic_pinggu_result = OrderedDict(zip(dic1.keys(), list_pinggu))
    for i in dic_pinggu_result.keys():
        d_test.setdefault(i.split('\t')[0],[])
        d_test[i.split('\t')[0]].append(dic_pinggu_result[i])
    
    
    for j, k in d_test.items():
        target = []
        for line in k:
            for z in line.split('，'):
                if '--' not in z:
                    target.append(z + '--' + line.split('--')[1])
                else:
                    target.append(z)
    
        dic_aa[j] = target
    
    
    for i, a in dic_aa.items():
        list_effect = []
        
        du = re.findall(r'毒副\w+--\w+', str(a))
        dai = re.findall(r'代谢\w+--\w+', str(a))
        liao = re.findall(r'疗效\w+--\w+', str(a))
        
        top_num_liao = re.findall(r'较好--\w+', str(liao))
        mid_num_liao = re.findall(r'适中--\w+', str(liao))
        bot_num_liao = re.findall(r'较差--\w+', str(liao))
        
        top_num_du = re.findall(r'较高--\w+', str(du))
        mid_num_du = re.findall(r'适中--\w+', str(du))
        bot_num_du = re.findall(r'较低--\w+', str(du))
        
        top_num_dai = re.findall(r'较快--\w+', str(dai))
        mid_num_dai = re.findall(r'适中--\w+', str(dai))
        bot_num_dai = re.findall(r'较慢--\w+', str(dai))
        
        
        
        #疗效结果判定
        if len(liao) == 1: #位点数目为1的时候
            if '适中' in liao[0] and level_dic[liao[0].split('--')[1]] >= level_dic['2A'] or  '较好' in liao[0]:
                list_effect.append('疗效较好')
            if '适中' in liao[0] and level_dic[liao[0].split('--')[1]] < level_dic['2A'] or  '较差' in liao[0]:
                list_effect.append('疗效适中')
        
        if len(liao) == 2:
            if len(top_num_liao) == 2 or len(top_num_liao ) == 1 and len(mid_num_liao ) == 1:
                list_effect.append('疗效较好')
            if len(top_num_liao) == 1 and len(bot_num_liao) == 1 or len(mid_num_liao) == 2 or len(mid_num_liao) == 1 and len(bot_num_liao) == 1 \
            or len(bot_num_liao) == 2:
                list_effect.append('疗效适中')
        
        if len(liao) == 3:
            if len(top_num_liao) == 3 or len(top_num_liao) == 2 and len(mid_num_liao) == 1 or len(top_num_liao) == 2 \
            and len(bot_num_liao) == 1 or len(top_num_liao) == 1 and len(mid_num_liao) ==2 or len(top_num_liao) == 1 \
            and len(mid_num_liao) == 1 and len(bot_num_liao) == 1:
                list_effect.append('疗效较好')
            if  len(top_num_liao) == 1 and len(bot_num_liao) == 2 or len(mid_num_liao) == 3 or len(mid_num_liao) == 2 \
            and len(bot_num_liao) == 1 or len(mid_num_liao) == 1 and len(bot_num_liao) == 2 or len(bot_num_liao) == 3:
                list_effect.append('疗效适中')
        
        
        
        
        #毒副结果判定
        if len(du) == 1:
            if '较高' in du[0]:
                list_effect.append('毒副较高')
            if '适中' in du[0]:
                list_effect.append('毒副适中')
            if '较低' in du[0]:
                list_effect.append('毒副较低')
        
        if len(du) == 2:    
            if len(top_num_du) == 2 or len(top_num_du) == 1 and len(mid_num_du) == 1 or len(top_num_du) == 1 and \
            level_dic[top_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 1:
                list_effect.append('毒副较高')
            if len(top_num_du) == 1 and level_dic[top_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) == 1 or len(mid_num_du) \
            == 2 or len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] >= level_dic['2A'] and len(bot_num_du) \
            == 1:
                list_effect.append('毒副适中')
            if len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] <= level_dic['2B'] and len(bot_num_du) == 1 \
            or len(bot_num_du) == 2:
                list_effect.append('毒副较低')
        
        if len(du) == 3:
            if len(top_num_du) == 3 or len(top_num_du) == 2 and len(mid_num_du) == 1 or len(top_num_du) == 1 and len(mid_num_du) \
            == 2 or len(top_num_du) == len(mid_num_du) == len(bot_num_du) == 1 or len(top_num_du) == 1 and \
            level_dic[top_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 2:
                list_effect.append('毒副较高')
            if len(top_num_du) == 1 and level_dic[top_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) \
            == 2 or len(mid_num_du) == 3 or len(mid_num_du) == 2 and len(bot_num_du) == 1 or len(mid_num_du) == 1 \
            and level_dic[mid_num_du[0].split('--')[1]] == level_dic['1A'] and len(bot_num_du) == 2:
                list_effect.append('毒副适中')
            if len(mid_num_du) == 1 and level_dic[mid_num_du[0].split('--')[1]] < level_dic['1A'] and len(bot_num_du) == 2 \
            or len(bot_num_du) == 3:
                list_effect.append('毒副较低')
        
        
        
        #代谢结果判定
        if len(dai) == 1:
            if '较快' in dai[0]:
                list_effect.append('代谢较快')
            if '适中' in dai[0]:
                list_effect.append('代谢适中')
            if '较慢' in dai[0]:
                list_effect.append('代谢较慢')
        
        if len(dai) == 2:
            if len(top_num_dai) == 2 or len(top_num_dai) == 1 and len(mid_num_dai) == 1:
                list_effect.append('代谢较快')
            if len(top_num_dai) == 1 and len(bot_num_dai) == 1 and level_dic[bot_num_dai[0].split('--')[1]] \
            < level_dic['1A'] or len(mid_num_dai) == 2:
                list_effect.append('代谢适中')
            if len(mid_num_dai) == 1 and len(bot_num_dai) == 1 or len(bot_num_dai) == 2 or len(top_num_dai) \
            == 1 and len(bot_num_dai) == 1 and level_dic[bot_num_dai[0].split('--')[1]] == level_dic['1A']:
                list_effect.append('代谢较慢')
    
    
        d_result[i] = '，'.join(list_effect)
        
    return d_result
                
        
            
            
            

In [37]:
def zz(client_name ):
    d_result = {}#临床提示映射
    list_pinggu = []
    d_test = {}
    dic_aa = {}
    for i in dic1.keys():
        list_pinggu.append(dic1[i][dic_client[client_name][i.split('\t')[2]]] + '--' + str(dic1[i]['证据等级']) )
        dic_pinggu_result = OrderedDict(zip(dic1.keys(), list_pinggu))
    for i in dic_pinggu_result.keys():
        d_test.setdefault(i.split('\t')[0],[])
        d_test[i.split('\t')[0]].append(dic_pinggu_result[i])
    
    
    
    for j, k in d_test.items():
        target = []
        for line in k:
            for z in line.split('，'):
                if '--' not in z:
                    target.append(z + '--' + line.split('--')[1])
                else:
                    target.append(z)
    
        dic_aa[j] = target
    return dic_aa

In [38]:
zz('张君')

{'表柔比星+氟尿嘧啶+奥沙利铂': ['疗效较好--3', '疗效较好--2A'],
 '顺铂+紫杉醇': ['疗效较差--2B'],
 '卡培他滨+紫杉醇': ['疗效较好--2A', '疗效较差--2B'],
 '紫杉醇': ['毒副适中--3', '疗效较差--2B'],
 '多西他赛': ['毒副较低--3', '代谢较快--3', '毒副较低--3', '代谢较慢--3'],
 '顺铂': ['疗效较差--2B', '毒副较高--2A', '疗效适中--2B', '毒副适中--2B'],
 '奥沙利铂': ['疗效较差--2B', '毒副较高--2A', '疗效适中--2B', '毒副适中--2B'],
 '卡培他滨': ['毒副较低--1A',
  '代谢较快--1A',
  '毒副较低--1A',
  '代谢较快--1A',
  '疗效较好--2A',
  '毒副较高--2A'],
 '氟尿嘧啶': ['毒副较低--1A',
  '代谢较快--1A',
  '毒副较低--1A',
  '代谢较快--1A',
  '疗效较好--2A',
  '毒副较高--2A',
  '疗效较好--3'],
 '替加氟': ['毒副较低--1A', '代谢较快--1A', '毒副较低--1A', '代谢较快--1A'],
 '伊立替康': ['毒副适中--2A', '毒副适中--2A']}

In [39]:
def gene_type_level(name):
    gene_type_dic = {}
    for i in dic1.keys():
        gene_type_dic.setdefault(i, {})
        gene_type_dic[i]['基因型'] = dic_client[name][i.split('\t')[-1]]
        gene_type_dic[i]['证据等级'] = dic1[i]['证据等级']
        text = dic1[i][dic_client[name][i.split('\t')[2]]]
        dai = re.findall(r'代谢', text)
        du = re.findall(r'毒副', text)
        liao = list(set(re.findall(r'疗效', text)))
        x = []
        for k in [liao, du, dai]:
            if k:
                x += k
        #print('，'.join(x).replace('副', '性'))
        
        

        gene_type_dic[i]['类别'] = '，'.join(x).replace('副', '性')
            
        
    return gene_type_dic

In [40]:
dic_info_client = {}
for i in range(df_client.shape[0]):
    dic_info_client.setdefault(df_client['姓名'][i], {})
    dic_info_client[df_client['姓名'][i]]['采样日期'] = str(df_client['采样时间'][i])[:4] + '-' + str(df_client['采样时间'][i])[4:6] + '-' + str(df_client['采样时间'][i])[6:8]
    dic_info_client[df_client['姓名'][i]]['收样日期'] = str(df_client['收样时间'][i])[:4] + '-' + str(df_client['收样时间'][i])[4:6] + '-' + str(df_client['收样时间'][i])[6:8]
    dic_info_client[df_client['姓名'][i]]['姓名'] = df_client['姓名'][i]
    dic_info_client[df_client['姓名'][i]]['性别'] = df_client['性别'][i]
    dic_info_client[df_client['姓名'][i]]['年龄'] = df_client['年龄'][i]
    dic_info_client[df_client['姓名'][i]]['样本编号'] = df_client['实验室编号'][i]
    dic_info_client[df_client['姓名'][i]]['临床提示'] = clinical_prompt(df_client['姓名'][i])
    dic_info_client[df_client['姓名'][i]]['基因型'] = gene_type_level(df_client['姓名'][i])
    dt = datetime.date.today()
    reportdate = '{}-{}-{}'.format(str(dt.year), str('%02d' % dt.month), str('%02d' % dt.day))
    dic_info_client[df_client['姓名'][i]]['报告日期'] = reportdate

In [41]:
def jr(name):
    result_list = []
    for drug in dic_info_client[name]['临床提示'].keys():
        judge_list = []
        result_dic = {}
        result_dic['Drugs'] = drug
        result_dic['Estimate'] = dic_info_client[name]['临床提示'][drug]
        result_dic['detail'] = []
        for sub_key, sub_value in dic_info_client[name]['基因型'].items():
            if drug == sub_key.split('\t')[0]:
                #detail_dic = {}
                if sub_key.split('\t')[1] not in judge_list:
                    judge_list.append(sub_key.split('\t')[1])
                    detail_dic = {}
                    detail_dic['Gene'] = sub_key.split('\t')[1]
                    detail_dic['new_detail'] = []
                    newdetail_dic = {}
                    newdetail_dic['rsid'] = sub_key.split('\t')[2]
                    newdetail_dic['GT'] = sub_value['基因型']
                    newdetail_dic['type'] = sub_value['类别']
                    newdetail_dic['level'] = sub_value['证据等级']
                    detail_dic['new_detail'].append(newdetail_dic)
                    result_dic['detail'].append(detail_dic)
                else:
                    newdetail_dic = {}
                    newdetail_dic['rsid'] = sub_key.split('\t')[2]
                    newdetail_dic['GT'] = sub_value['基因型']
                    newdetail_dic['type'] = sub_value['类别']
                    newdetail_dic['level'] = sub_value['证据等级']
                    for i in result_dic['detail']:
                        if i['Gene'] == sub_key.split('\t')[1]:
                            i['new_detail'].append(newdetail_dic)
                    #result_dic['detail'].append(detail_dic)
        result_list.append(result_dic)
    return result_list

In [42]:
finall_dic = {}
for i in range(df_client.shape[0]):
    finall_dic.setdefault(df_client['姓名'][i], {})
    if str(df_client['采样时间'][i]) == '***':
        finall_dic[df_client['姓名'][i]]['采样日期'] = '***'
    else:
    
        finall_dic[df_client['姓名'][i]]['采样日期'] = str(df_client['采样时间'][i])[:4] + '-' + str(df_client['采样时间'][i])[4:6] + '-' + str(df_client['采样时间'][i])[6:8]
    finall_dic[df_client['姓名'][i]]['收样日期'] = str(df_client['收样时间'][i])[:4] + '-' + str(df_client['收样时间'][i])[4:6] + '-' + str(df_client['收样时间'][i])[6:8]
    finall_dic[df_client['姓名'][i]]['姓名'] = df_client['姓名'][i]
    finall_dic[df_client['姓名'][i]]['性别'] = df_client['性别'][i]
    finall_dic[df_client['姓名'][i]]['年龄'] = df_client['年龄'][i]
    finall_dic[df_client['姓名'][i]]['样本编号'] = df_client['实验室编号'][i]
    dt = datetime.date.today()
    reportdate = '{}-{}-{}'.format(str(dt.year), str('%02d' % dt.month), str('%02d' % dt.day))
    finall_dic[df_client['姓名'][i]]['报告日期'] = reportdate
    finall_dic[df_client['姓名'][i]]['Result'] =  jr(df_client['姓名'][i])
    finall_dic[df_client['姓名'][i]]['临床诊断'] = df_client['临床诊断'][i]

In [43]:
file = 'template.docx'
file_date = str(dt.year)+ str('%02d' % dt.month)+ str('%02d' % dt.day)
#tpl = DocxTemplate(file)
cancer_name = '胃癌化疗用药报告'
for name in finall_dic.keys():
    #outdoc = re.sub('template',cancer_name + '_' + name + '_' + file_date , file)
    outdoc = re.sub('template', name + '-' + '胃癌化疗用药检测报告' , file)
    tpl = DocxTemplate(file)
    tpl.render(finall_dic[name])
    tpl.save(outdoc)